In [1]:
from llama_index import GPTVectorStoreIndex, download_loader, SimpleDirectoryReader, LLMPredictor, PromptHelper,ServiceContext
from langchain.agents import initialize_agent, Tool
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
import openai
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.embeddings import OpenAIEmbeddings
from langchain.agents import create_pandas_dataframe_agent
from llama_index import LangchainEmbedding
import os
import pandas as pd

In [6]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://openaidemo-hu.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "XXX"


openai.api_type = "azure"
openai.api_version = "2023-05-15"
openai.api_base = "https://openaidemo-hu.openai.azure.com/"
openai.api_key = "XXX"

davinci = AzureOpenAI(
    deployment_name="text-davinci-003",
    model_name="text-davinci-003",
    temperature=0
)

gpt4 = AzureChatOpenAI(
    openai_api_base=openai.api_base,
    openai_api_version="2023-05-15",
    deployment_name="gpt4",
    openai_api_key=openai.api_key,
    openai_api_type=openai.api_type,
    temperature=0
)


embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1, openai_api_version="2023-05-15")

In [7]:
# 读取CSV文件
data = pd.read_csv('sample_data_title_embedding.csv')

In [8]:
priorityEntityData = pd.read_csv("priority_entity.csv")
priorityEntityData['similarity'] = priorityEntityData['priority_entity'].apply(lambda x: embeddings.embed_query(x))

priorityEntityData.to_csv("priority_entity_embedding.csv")

In [9]:
priorityEntityDataEmbedding = pd.read_csv('priority_entity_embedding.csv')

In [10]:
import numpy as np

def cosine_similarity_calculation(x_vector, y_vector):
    # 计算点积
    dot_product = np.dot(x_vector, y_vector)
    # 计算向量的范数（长度）
    norm_array1 = np.linalg.norm(x_vector)
    norm_array2 = np.linalg.norm(y_vector)
    return dot_product / (norm_array1 * norm_array2)

def convet_to_float_array(input: str) -> np.array:
    values_str = input.strip('[]')
    values_list = values_str.split(',')
    # 将字符串列表转换为浮点数数组
    float_array = np.array([float(value) for value in values_list])
    return float_array

def tilte_query_cosine_similarity_calculation(input_vector_str:str,query_vector):
    float_array_result= convet_to_float_array(input_vector_str)
    return cosine_similarity_calculation(query_vector,float_array_result)

for index,record in priorityEntityDataEmbedding.iterrows():
    
    column_name = "similarity" + "_" + str(record["No."])
    query_vector = convet_to_float_array(record["similarity"])
    data[column_name] = data['title_embedding'].apply(lambda x: tilte_query_cosine_similarity_calculation(x,query_vector))
 



data.to_csv('sample_data_title_embedding_priorityEntity_similar.csv',index=False)


同样地，计算出来了余弦相似性便可以取得，余弦阈值的所有记录，同样用0.85为例子（经验值，看到几个标题大概是0.86,0.87左右，所以0.9不合适）

In [11]:
data = pd.read_csv('sample_data_title_embedding_priorityEntity_similar.csv')

In [12]:
# 选择包含'similarity'的列
similarity_cols = data.filter(like='similarity').columns

priorityEntityEventSourcelist = []
# 选择值大于0.8的所有记录
for col in similarity_cols:
    matched_title_df = data.loc[data[col].gt(0.85)]
    num =0
    eventSource=""
    for index,record in matched_title_df.iterrows():
        num+=1
        rowNo = str(num) + ". "
        eventSource = eventSource + rowNo + record["title"] + "\n "
    priorityEntityEventSourcelist.append(eventSource)

display(priorityEntityEventSourcelist)


['1. 小米第5年上榜世界500强 雷军感谢用户信任\n 2. 雷军：小米连续5年上榜财富世界500强\n 3. 大行评级丨瑞银：予小米16港元目标价 为中国智能手机领域首选\n 4. 大行评级丨瑞银：予小米16港元目标价 为中国智能手机领域首选\n ',
 '1. 那个能打的蔚来，又回来了\n ']

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=('''You are a financial journalist who, based on the financial news headlines provided by the user, Summarize into a simple, clear, easily understandable sentence that grabs attention in fewer than 15 words. 
            if the language of headlines is Chinses,please answer in Chinese. ''')
        ),
        HumanMessagePromptTemplate.from_template("The corresponding financial news headlines are as follows: {event_Source}"),
    ]
)

for priorityEntityEventSource in priorityEntityEventSourcelist:
    eventFinal = gpt4(template.format_messages(event_Source=priorityEntityEventSource)).content
    print(eventFinal)
    

小米连续五年跻身世界500强，瑞银给予16港元目标价。
蔚来强势回归，再度引领竞争。
